# RoleML Helloworld

This notebook contains a minimal application of RoleML.

## Basic Concepts

In RoleML, **roles** are functional components that interact with each other within a DML architecture. Each role encapsulates a set of message _channels_ that serve as interfaces for other roles. A _DML architecture_ can be expressed by a couple of roles. The most common roles include Trainer (to maintain local model and perform training), Aggregator (to perform model aggregation) and Coordinator (to define the overall workflow of the server or an independent client).

There are three types of channels: _services_ and tasks stand for synchronous and asynchronous function calls respectively, which are used in directional communication. Events follow the publish-subscribe model and are used for non-directional communication. All channels defined in a role should focus on a single responsibility such as training or aggregation.

Sometimes a role needs to interact with different types of roles. A new abstraction called _relationship_ is therefore introduced to manage this. Each relationship is represented by a name and mapped to a list of instantiated roles on runtime. For example, a relationship named trainers may contain a list of Trainer instances on different clients, from which a Client Selector can select participants. Relationships are configured at the node level and only the names will be referenced by the roles.

## A Minimal Application

Step 1 - import RoleML:

In [ ]:
import roleml.essentials as rml
from roleml.kits.interfaces import Runnable

Step 2 - define roles. Here we define a role with a service channel and use another role to call this service:

In [ ]:
class Helloworld(rml.Role):

    @rml.Service()
    def echo(self, caller, args, payloads):
        return 'hello world!'

In [ ]:
class Player(rml.Role, Runnable):

    def run(self):
        """ Runnable.run() will be automatically executed when deployed on an actor """
        print(self.call('helloworld', 'echo'))

Step 3 - write a configuration to put these roles in an actor:

In [ ]:
conf = {
    'name': 'roleml-abc',
    'address': '127.0.0.1:5000',
    'roles': {
        'helloworld': {
            'class': '__main__.Helloworld'
        },
        'player': {
            'class': '__main__.Player'
        }
    }
}   # type: rml.ActorBootstrapSpec

Step 4 - build the actor and run it!

**The actor will just keep running. if you can see the helloworld output, you may stop running the cell to stop the actor.**

In [ ]:
builder = rml.ActorBuilder()
builder.load_config(conf)
actor = builder.build()
actor.run()     # will not stop unless manually interrupted

## Make It Distributed

Let's try to put the two roles in different actors:

In [ ]:
from roleml.extensions.messaging.invokers.requests import RequestsProcedureInvoker
from roleml.extensions.messaging.providers.flask import FlaskProcedureProvider

In [ ]:
conf_helloworld = {
    'name': 'a1',
    'address': '127.0.0.1:5001',
    # 'procedure_invoker': RequestsProcedureInvoker,
    'procedure_provider': FlaskProcedureProvider,
    'contacts': {
        'a2': '127.0.0.1:5002'
    },
    'roles': {
        'helloworld': {
            'class': '__main__.Helloworld'
        }
    }
}   # type: rml.ActorBootstrapSpec

In [ ]:
conf_player = {
    'name': 'a2',
    'address': '127.0.0.1:5002',
    'procedure_invoker': RequestsProcedureInvoker,
    # 'procedure_provider': FlaskProcedureProvider,
    'roles': {
        'player': {
            'class': '__main__.Player'
        }
    },
    'contacts': {
        'a1': '127.0.0.1:5001'
    },
    'relationships': {
        'helloworld': ['a1/helloworld']
    }
}   # type: rml.ActorBootstrapSpec

Since `Player` needs to call the service in `Helloworld`, actor `a2` needs to know the network address of `a1`. Meanwhile, the service provider side needs to know who calls the service (reflected in the `caller` argument which represents the calling role instance), and therefore actor `a1` also needs to know the network address of `a2`.

The Helloworld role states that it needs to call a service on a role that belong to the relationship `helloworld`. When both roles are in the same actor, we don't need to manually configure the relationship because the actor will fall back to find a local role instance named `helloworld`. However, when both roles are in different actors, the relationship must be configured. Note that relationships are directional - `a1` does not necessarily need to know what roles on `a2` mean to it.

Now we start both actors in separate threads:

In [ ]:
from threading import Thread

In [ ]:
builder = rml.ActorBuilder()
builder.load_config(conf_helloworld)
a1 = builder.build()
t1 = Thread(target=a1.run)
t1.start()

In [ ]:
builder = rml.ActorBuilder()
builder.load_config(conf_player)
a2 = builder.build()
t2 = Thread(target=a2.run)
t2.start()

If everything goes fine, you should see the helloworld output. Finally, let's stop the actors gracefully:

In [ ]:
a1.stop()
a2.stop()

## What's Next?

[RoleML in 100 minutes](./LEARN.ipynb) is a Jupyter notebook that will introduce more detail of RoleML in a case study. Also check other documents in the `docs` directory of the source code archive.